In [ ]:
basedir = '/home/abhinavgupta0110/NeuralODEs_ROM_Closure'

import os

is_google_colab = False
is_use_GPU = False

### Mount the Google drive if needed

In [ ]:
if is_use_GPU:
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('No GPU found!')
    else:
        print(gpu_info)

if is_google_colab:
    from google.colab import drive
    drive.mount('/content/drive')

    %pip install quadpy
    
os.chdir(os.path.join(basedir, 'neuralDDE_ROM_Closure'))

### Load modules

In [ ]:
from src.utilities.DDE_Solver import ddeinttf 
import src.solvers.neuralDDE_with_adjoint as ndde
import src.advec_diff_case.advec_diff_eqn as adeq
import src.advec_diff_case.rom_advec_diff as rom

import time
import sys
from IPython.core.debugger import set_trace

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from shutil import move
import pickle

tf.keras.backend.set_floatx('float32')
import logging
tf.get_logger().setLevel(logging.ERROR)

## Define some useful classes

### Class for user-defined arguments

In [ ]:
class rom_ad_eq_nDDE_args(ndde.arguments, rom.rom_eqn_args):

    def __init__(self, batch_time = 12, batch_time_skip = 2, batch_size = 5, epochs = 500, learning_rate = 0.05, decay_rate = 0.95, test_freq = 1, plot_freq = 2, 
                 d_max = 1.1, rnn_nmax = 3, rnn_dt = 0.5, state_dim = 2, adj_data_size = 2,
                 model_dir = 'ROM_nODE_testcase/model_dir_test', restart = 0, val_percentage = 0.2,
                 T = 2., nt = 200, L = 1., nx = 100, Re = 250, u_bc_0 = 0., u_bc_L =0., rom_dims = 2, isplot = True, reg_coeff = 0.001): # add more arguments as needed
        
        ndde.arguments.__init__(self, data_size = nt, batch_time = batch_time, batch_time_skip = batch_time_skip, batch_size = batch_size, epochs = epochs,
                           learning_rate = learning_rate, decay_rate = decay_rate, test_freq = test_freq, plot_freq = plot_freq, d_max = d_max, rnn_nmax = rnn_nmax, 
                           rnn_dt = rnn_dt, state_dim = rom_dims, adj_data_size = rom_dims, model_dir = model_dir, restart = restart, val_percentage = val_percentage, isplot = isplot)

        rom.rom_eqn_args.__init__(self, T = T, nt = nt, L = L, nx = nx, Re = Re, u_bc_0 = u_bc_0, u_bc_L = u_bc_L, rom_dims = rom_dims, 
                              rom_batch_size = batch_size, ad_eq_batch_size = 1)
        
        self.rom_args_for_plot = rom.rom_eqn_args(T = T, nt = nt, L = L, nx = nx, Re = Re, u_bc_0 = u_bc_0, u_bc_L = u_bc_L, rom_dims = rom_dims, 
                              rom_batch_size = 1, ad_eq_batch_size = 1)
        
        self.reg_coeff = reg_coeff

### Create modes using the FOM solution

In [ ]:
class create_mean_modes:
    def __init__(self, fom_sol, app, t):
        self.fom_sol = fom_sol
        self.app = app
        self.t = t

    def __call__(self):

        u_analy = tf.transpose(self.fom_sol)

        u_mean = tf.expand_dims(tf.reduce_mean(u_analy, axis=-1), axis=1)

        S, U, V = tf.linalg.svd(u_analy - tf.tile(u_mean, [1, len(self.t)]))

        ui = U[:, 0:self.app.rom_dims]

        return u_mean, ui

### Define the neural net architecture

In [ ]:
class DDEFunc(tf.keras.Model):

    def __init__(self, rom_model, **kwargs):
        super(DDEFunc, self).__init__(**kwargs)
        
        self.x1 = tf.keras.layers.Dense(5, activation='tanh',
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.x2 = tf.keras.layers.Dense(5, activation='tanh',
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.x3 = tf.keras.layers.Dense(5, activation='tanh',
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.x4 = tf.keras.layers.Dense(5, activation='tanh',
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.x5 = tf.keras.layers.Dense(5, activation='tanh',
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.out = tf.keras.layers.Dense(args.state_dim, activation='linear',
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.rom_model = rom_model


    def call(self, y, t, d):
        y_nn = y(t)
        for i in range(len(self.layers)):
            y_nn = self.layers[i](y_nn)
        return y_nn + self.rom_model(y, t) + args.reg_coeff * tf.random.normal([y_nn.shape[0], args.state_dim])

### Define a custom loss function

In [ ]:
class custom_loss(tf.keras.losses.Loss):

    def call(self, true_y, pred_y):
        loss = tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.math.squared_difference(pred_y, true_y), axis=-1)), axis=0)
        return loss

### Define a custom plotting function

In [ ]:
class custom_plot:

    def __init__(self, true_y, true_y_red_ic, y_no_nn, t, figsave_dir, args):
        self.true_y = true_y
        self.true_y_red_ic = true_y_red_ic
        self.y_no_nn = y_no_nn
        self.t = t
        self.figsave_dir = figsave_dir
        self.args = args
        self.colors = ['b', 'g', 'r', 'k', 'c', 'm']

    def plot(self, *pred_y, epoch = 0):
        fig = plt.figure(figsize=(6, 4), facecolor='white')
        ax = fig.add_subplot(111)

        ax.cla()
        ax.set_title('Trajectories')
        ax.set_xlabel('t')
        ax.set_ylabel('$a_i$')
        ax.set_xlim(min(self.t.numpy()), max(self.t.numpy()))
        ax.set_ylim(-1, 1.)

        for i in range(self.args.rom_dims):
            ax.plot(self.t.numpy(), self.true_y.numpy()[:, 0, i], self.colors[i % self.args.rom_dims]+'-', label = 'True mode '+str(i+1))
            ax.plot(self.t.numpy(), self.true_y_red_ic.numpy()[:, 0, i], self.colors[i % self.args.rom_dims]+':', label = 'True-Red. IC mode '+str(i+1))
            ax.plot(self.t.numpy(), self.y_no_nn.numpy()[:, 0, i], self.colors[i % self.args.rom_dims]+'-.', label = 'GP mode '+str(i+1))
            if epoch != 0 or self.args.restart == 1 :
                ax.plot(self.t.numpy(), pred_y[0].numpy()[:, 0, i], self.colors[i % self.args.rom_dims]+'--', label = 'Learned mode '+str(i+1))

        ax.legend(bbox_to_anchor=(1.04,1), loc="upper left")
        plt.show() 

        if epoch != 0: 
            fig.savefig(os.path.join(self.figsave_dir, 'img'+str(epoch)))

### Initial Conditions

In [ ]:
class initial_cond:

    def __init__(self, x, app):
        self.x = x
        self.app = app

    def __call__(self, t):
        u0 = self.x / (1. + np.sqrt(1./self.app.t0) * np.exp(self.app.Re * self.x**2 / 4., dtype = np.float64))
        return tf.convert_to_tensor([u0], dtype=tf.float32)

In [ ]:
class red_initial_cond:
    
    def __init__(self, ai_t0, u_mean, u_modes):
        self.ai_t0 = ai_t0
        self.u_mean = u_mean
        self.u_modes = u_modes
        
    def __call__(self, t):

        u0_rom = tf.transpose(self.u_mean, perm=[1, 0]) \
            + tf.cast(tf.einsum('ab, db -> da', tf.cast(self.u_modes, tf.float64), tf.cast(self.ai_t0(t), tf.float64)), tf.float32)
        
        return u0_rom

### Initialize model related parameters

In [ ]:
args = rom_ad_eq_nDDE_args(batch_time = 6, batch_time_skip = 2, batch_size = 2, epochs = 500, learning_rate = 0.075, decay_rate = 0.97, test_freq = 1, plot_freq = 2,
                    d_max = 0.2, rnn_nmax = 1, rnn_dt = 0.1, model_dir = 'ROM_nODE_testcase/model_dir_case', restart = 0, val_percentage = 1.,
                    T = 1.25, nt = 125, L = 1., nx = 100, Re = 1000, u_bc_0 = 0., u_bc_L =0., rom_dims = 3, reg_coeff = 0.0)

### Make a copy of the current script

In [ ]:
os.chdir(basedir)

if not os.path.exists(args.model_dir):
    os.makedirs(args.model_dir)

checkpoint_dir = os.path.join(args.model_dir, "ckpt")
checkpoint_prefix = os.path.join(checkpoint_dir, "cp-{epoch:04d}.ckpt")
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

figsave_dir = os.path.join(args.model_dir, "img")
if not os.path.exists(figsave_dir):
    os.makedirs(figsave_dir)

!jupyter nbconvert --to python neuralDDE_ROM_Closure/testcases/AD_Eqn_ROM/neuralODE_ROM_AD_Eqn_TestCase-Reduced_IC.ipynb
move("neuralDDE_ROM_Closure/testcases/AD_Eqn_ROM/neuralODE_ROM_AD_Eqn_TestCase-Reduced_IC.py", os.path.join(args.model_dir, "orig_run_file.py"))

with open(os.path.join(args.model_dir, 'args.pkl'), 'wb') as output:
    pickle.dump(args, output, pickle.HIGHEST_PROTOCOL)

### Solve for the full-order-model

In [ ]:
x = tf.linspace(0., args.L, args.nx)
t = tf.linspace(0., args.T, args.nt) # Time array

u0 = initial_cond(x, args) # Initial conditions

op = adeq.operators(args)

u_fom = ddeinttf(adeq.ad_eqn(op), u0, t)

# Compute FOM for the validation time
dt = t[1] - t[0]
val_t_len =  args.val_percentage * (t[-1] - t[0])
n_val = np.ceil(np.abs(val_t_len/dt.numpy())).astype(int)
val_t = tf.linspace(t[-1], t[-1] + val_t_len, n_val)

val_u_fom = ddeinttf(adeq.ad_eqn(op), ndde.create_interpolator(u_fom, t), val_t)

print('FOM done!')

### Project FOM on the modes

In [ ]:
# Create modes for the training and validation period combined
u_mean, ui = create_mean_modes(tf.squeeze(tf.concat([u_fom, val_u_fom], axis=0), axis=1), args, tf.concat([t, val_t], axis=0))()

ai_t0 = rom.initial_cond_rom(u0, ui, u_mean)

true_ai = u_fom - tf.tile(tf.expand_dims(tf.transpose(u_mean, perm=[1, 0]), axis=0), [args.nt, args.multi_solve_size, 1])
true_ai = tf.cast(tf.einsum('ab, cda -> cdb', tf.cast(ui, tf.float64), tf.cast(true_ai, tf.float64)), tf.float32)


Solve the ROM model

In [ ]:
true_rom_model = rom.rom_ad_eqn(um = u_mean, ui = ui, op = op, app = args.rom_args_for_plot)
ai_whole = ddeinttf(true_rom_model, ai_t0, tf.concat([t, val_t], axis=0))

#### Create validation set

In [ ]:
val_obj = ndde.create_validation_set(ai_t0, t, args)

ai, val_ai = val_obj.data_split(ai_whole)

val_true_ai = val_u_fom - tf.tile(tf.expand_dims(tf.transpose(u_mean, perm=[1, 0]), axis=0), [val_obj.val_t.shape[0], args.multi_solve_size, 1])
val_true_ai = tf.cast(tf.einsum('ab, cda -> cdb', tf.cast(ui, tf.float64), tf.cast(val_true_ai, tf.float64)), tf.float32)

In [ ]:
u0_red = red_initial_cond(ai_t0, u_mean, ui)

u_fom_red_ic = ddeinttf(adeq.ad_eqn(op), u0_red, tf.concat([t, val_t], axis=0))

In [ ]:
true_ai_red = u_fom_red_ic - tf.tile(tf.expand_dims(tf.transpose(u_mean, perm=[1, 0]), axis=0), [args.nt + n_val, args.multi_solve_size, 1])
true_ai_red = tf.cast(tf.einsum('ab, cda -> cdb', tf.cast(ui, tf.float64), tf.cast(true_ai_red, tf.float64)), tf.float32)

true_ai_red, val_true_ai_red = val_obj.data_split(true_ai_red)

## Main part starts here

### Make objects and define learning-rate schedule

In [ ]:
time_meter = ndde.RunningAverageMeter(0.97)

func = DDEFunc(rom.rom_ad_eqn(um = u_mean, ui = ui, op = op, app = args))
adj_func = ndde.adj_eqn_ODE(func, args)
get_batch = ndde.create_batch(true_ai_red, ai_t0, t, args)
loss_obj = custom_loss()
plot_obj = custom_plot(tf.concat([true_ai, val_true_ai], axis=0), tf.concat([true_ai_red, val_true_ai_red], axis=0), tf.concat([ai, val_ai], axis=0), 
                       tf.concat([t, val_t], axis=0), figsave_dir, args)
loss_history = ndde.history(args)

initial_learning_rate = args.learning_rate
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=args.niters,
    decay_rate=args.decay_rate,
    staircase=True)

### Quick test to see how the true coefficients looks like

In [ ]:
if args.restart == 1: 
    func.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
    pred_y = ddeinttf(func, ai_t0, tf.concat([t, val_t], axis=0), fargs=([args.rnn_nmax, args.rnn_dt],))
    
    plot_obj.plot(pred_y, epoch = 0)

    loss_history.read()
    
    initial_learning_rate = 0.05
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps=args.niters, decay_rate=0.95, staircase=True)
    
else:
    plot_obj.plot(epoch = 0)

### Training starts here

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate = lr_schedule)

nDDE_train_obj = ndde.train_nDDE(func = func, adj_func = adj_func, d = [args.rnn_nmax, args.rnn_dt], loss_obj = loss_obj, batch_obj = get_batch,
                            optimizer = optimizer, args = args, plot_obj = plot_obj, time_meter = time_meter, checkpoint_dir = checkpoint_prefix, 
                            validation_obj = val_obj, loss_history_obj = loss_history)

nDDE_train_obj.train(true_ai_red, ai_t0, t, val_true_ai_red)